# QR Factorization 

AUTHOR 

MD SHORIFUL ISLAM;   
EMAIL- ISLAM5@BYU.EDU

# Introduction 
In linear algebra, a QR factorization of a matrix is a factorization of a matrix A into a product $A = QR$ of an orthogonal matrix Q and an upper triangular matrix R. QR factorization is often used to solve the linear least squares problem and is the basis for a particular eigenvalue algorithm, the QR algorithm.
One of the key benefits of using QR factorization over other methods for solving linear least squares is that it is more numerically stable, albeit at the expense of being slower to execute. Hence if we are performing a large quantity of regressions as part of a trading backtest, for instance, we need to consider very extensively whether QR factorization is the best fit.

For a square matrix A the QR factorization converts A into the product of an orthogonal matrix Q (i.e. $QTQ=I$) and an upper triangular matrix R. Hence:

$A=QR$
The first step is to create the vector x, which is the k-th column of the matrix A, for step k. We define $α=−sgn(xk)(||x||)$. The norm $||⋅||$ used here is the Euclidean norm. Given the first column vector of the identity matrix, I of equal size to A, $e1=(1,0,...,0)T$, we create the vector u:

$u=x+αe1$
Once we have the vector u, we need to convert it to a unit vector, which we denote as v:

$v=u/||u||$
Now we form the matrix Q out of the identity matrix I and the vector multiplication of v:

$Q=I−2vvT$
Q is now an m×m Householder matrix, with $Qx=(α,0,...,0)T$. We will use Q to transform A to upper triangular form, giving us the matrix R. We denote Q as Qk and, since k=1 in this first step, we have Q1 as our first Householder matrix. 
Hence, we define Qk as the block matrix $Q_{k}$:

\begin{bmatrix}
I_{k-1} & 0\\
0 & Q_{k}
\end{bmatrix}

$Q_{k}=(I_{k}−100Q′_{k})$
Once we have carried out t iterations of this process we have R as an upper triangular matrix:

$R=Q_{t}...Q_{2}Q_{1}A$
Q is then fully defined as the multiplication of the transposes of each Qk:

$Q=Q^{T}_{1}Q^{T}_{2}...Q^{T}_{t}$

This gives $A=QR$, the QR factorization of A.

In [26]:
#Finding the QR factorization using SciPy
#Print A, Q and R at the output
import pprint

In [27]:
import scipy

In [28]:
import scipy.linalg   # SciPy Linear Algebra Library

In [29]:
#Create an matrix
A = scipy.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])  # From the Wikipedia Article on QR factorization
Q, R = scipy.linalg.qr(A)

In [30]:
#print A matrix
print ("A:")
#Print the formatted represntation of A on stream, followed by a newline
pprint.pprint(A)

A:
array([[ 12, -51,   4],
       [  6, 167, -68],
       [ -4,  24, -41]])


In [8]:
print ("Q:")
pprint.pprint(Q)

Q:
array([[-0.85714286,  0.39428571,  0.33142857],
       [-0.42857143, -0.90285714, -0.03428571],
       [ 0.28571429, -0.17142857,  0.94285714]])


In [10]:
print ("R:")
pprint.pprint(R)

R:
array([[ -14.,  -21.,   14.],
       [   0., -175.,   70.],
       [   0.,    0.,  -35.]])


# Python implementation of QR factorization without any library 

In [11]:
from math import sqrt

In [12]:
from pprint import pprint

In [17]:
def mult_matrix(M, N):
    """Multiply square matrices of same dimension M and N"""
    # Converts N into a list of tuples of columns                                                                     
    tuple_N = zip(*N)
        # Nested list comprehension to calculate matrix multiplication                                                    
    return [[sum(el_m * el_n for el_m, el_n in zip(row_m, col_n)) for col_n in tuple_N] for row_m in M]

In [18]:
def trans_matrix(M):
    """Take the transpose of a matrix."""
    n = len(M)
    return [[ M[i][j] for i in range(n)] for j in range(n)]


In [19]:
def norm(x):
    """Return the Euclidean norm of the vector x."""
    return sqrt(sum([x_i**2 for x_i in x]))


In [20]:
def Q_i(Q_min, i, j, k):
    """Construct the Q_t matrix by left-top padding the matrix Q                                                      
    with elements from the identity matrix."""
    if i < k or j < k:
        return float(i == j)
    else:
        return Q_min[i-k][j-k]


In [21]:
def householder(A):
    """Performs a Householder Reflections based QR Decomposition of the                                               
    matrix A. The function returns Q, an orthogonal matrix and R, an                                                  
    upper triangular matrix such that A = QR."""
    n = len(A)

    # Set R equal to A, and create Q as a zero matrix of the same size
    R = A
    Q = [[0.0] * n for i in xrange(n)]

    # The Householder procedure
    for k in range(n-1):  # We don't perform the procedure on a 1x1 matrix, so we reduce the index by 1
        # Create identity matrix of same size as A                                                                    
        I = [[float(i == j) for i in xrange(n)] for j in xrange(n)]

        # Create the vectors x, e and the scalar alpha
        # Python does not have a sgn function, so we use cmp instead
        x = [row[k] for row in R[k:]]
        e = [row[k] for row in I[k:]]
        alpha = -cmp(x[0],0) * norm(x)

        # Using anonymous functions, we create u and v
        u = map(lambda p,q: p + alpha * q, x, e)
        norm_u = norm(u)
        v = map(lambda p: p/norm_u, u)

        # Create the Q minor matrix
        Q_min = [ [float(i==j) - 2.0 * v[i] * v[j] for i in xrange(n-k)] for j in xrange(n-k) ]

        # "Pad out" the Q minor matrix with elements from the identity
        Q_t = [[ Q_i(Q_min,i,j,k) for i in xrange(n)] for j in xrange(n)]

        # If this is the first run through, right multiply by A,
        # else right multiply by Q
        if k == 0:
            Q = Q_t
            R = mult_matrix(Q_t,A)
        else:
            Q = mult_matrix(Q_t,Q)
            R = mult_matrix(Q_t,R)

    # Since Q is defined as the product of transposes of Q_t,
    # we need to take the transpose upon returning it
    return trans_matrix(Q), R

In [22]:
#creating a matrix 
A = [[12, -51, 4], [6, 167, -68], [-4, 24, -41]]
#finding the QR factorization
Q, R = householder(A)

NameError: name 'xrange' is not defined

In [23]:
print ("A:")
pprint(A)

A:
[[12, -51, 4], [6, 167, -68], [-4, 24, -41]]


In [24]:
print ("Q:")
pprint(Q)

Q:
array([[-0.85714286,  0.39428571,  0.33142857],
       [-0.42857143, -0.90285714, -0.03428571],
       [ 0.28571429, -0.17142857,  0.94285714]])


In [25]:
print ("R:")
pprint(R)

R:
array([[ -14.,  -21.,   14.],
       [   0., -175.,   70.],
       [   0.,    0.,  -35.]])


# Application of QR Factorization 

# Linear Equations

In numerical analysis, different factorization are used to implement efficient matrix algorithms. For instance, when solving a system of linear equations , the matrix A can be factorized via the LU factorization. The LU factorization factorizes a matrix into a lower triangular matrix L and an upper triangular matrix U. The systems and require fewer additions and multiplications to solve, compared with the original system , though one might require significantly more digits in inexact arithmetic such as floating point.
Similarly, the QR factorization expresses A as QR with Q a unitary matrix and R an upper triangular matrix. The system $Q(Rx) = b$ is solved by $Rx = Q^{T}b = c$, and the system $Rx = c$ is solved by 'back substitution'. The number of additions and multiplications required is about twice that of using the LU solver, but no more digits are required in inexact arithmetic because the QR factorization is numerically stable.

# Eigenvalues

Even though QR factorization can be used to solve a system of Linear Equations and is in fact better than Gaussian Elimination in terms of stability, it is rarely used for the same because of its cost of computation being twice that of Gaussian elimination.
Another important use of QR factorization is inspiring the QR algorithm for calculation of eigen value factorization.

# Conclusion 
The result of the python generated code and the SciPy code should be same for the accurate python code. I got the same result for the python code and SciPy. This code is valid only both for the Python2 and Python3. It may show some error in python3 because python3 prefers range instead of xrange command. xrange need to be change in range for python3 which is a valid command to avoid any kind of error. 